# Prototype pour la recherche de nutri-score
L'excution de ce notebook nécessite l'utilisation de [Voilà](https://github.com/voila-dashboards/voila) pour une utilisation optimale.

Les données sont celles qui ont été générées avec le notebook **P3_01_notebook_clean**

## Chargement des bibliothèques

In [1]:
# Chargement des bibliothèques
import numpy as np
import pandas as pd

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

from ipywidgets import HBox, VBox, FloatSlider, interactive_output
import ipywidgets as widget
from IPython.display import display

## Chargement des données

In [2]:
# Répertoire pour les données
data_dir = "../data/"

In [3]:
data = pd.read_csv(data_dir + "fr.openfoodfacts_clean.csv", sep=',',low_memory=False)

## Fonction de recherche du nutri-score le plus approchant

In [4]:
def find_nutri_score(fat,carbohydrate, fiber, protein, salt):
    """
    Recherche le nutri-csore le plus proche en fonction des paramètres fournis    
    """
    threshold = 1 
    grade = data[
        (data['fat_100g'] > (fat - threshold)) & (data['fat_100g'] < (fat + threshold)) &
        (data['fiber_100g'] > (fiber - threshold)) & (data['fiber_100g'] < (fiber + threshold)) &
        (data['carbohydrates_100g'] > (carbohydrate - threshold)) & (data['carbohydrates_100g'] < (carbohydrate + threshold))                
    ]
    if (fat + carbohydrate + fiber + protein + salt) == 0:
        return 

    display(widget.HTML("<h2>Resultats</h2>"))
    
    
    try:        
        nutri_score = grade["nutrition_grade_fr"].mode().iat[0]
        
        image = "img/ns_" + nutri_score.upper() + ".png"                
        #print(f"Nutri-score : {nutri_score.upper()}")
        img = mpimg.imread(image)
        plt.imshow(img)
        plt.axis('off')
        plt.show()
        examples = grade[grade["nutrition_grade_fr"] == nutri_score].head(5)       
        display(widget.HTML("<h3>Quelques produits similaires</h3>"))
        display(widget.HTML("<ul style='list-style: square inside;'>"))
        for index, row in examples.iterrows():            
            display(widget.HTML(f"<li>{row['product_name']} ({row['brands']})</li>"))            
        display(widget.HTML("</ul>"))
    except IndexError:
        display(widget.HTML("<div style='color:red'>Impossible de déterminer le nutri-score</div>"))
    

## Définition des variables qui seront saisies par l'utilisateur

In [5]:
# Définition des variables
fat = FloatSlider(description='Lipides', value=0, min=0, max=100, step=.5)
carbohydrate = FloatSlider(description='Glucides', value=0, min=0, max=100, step=.5)
fiber = FloatSlider(description='Fibres', value=0, min=0, max=100, step=.5)
protein = FloatSlider(description='Protéines', value=0, min=0, max=100, step=.5)
salt = FloatSlider(description='Sel', value=0, min=0, max=100, step=.5)

## Affichage de la page web

In [6]:
# Affichage
display(widget.HTML("<h1>Recherche de Nutri-score</h1>"))
display(widget.HTML("<div>Saisissez les Lipides, glucides, fibres, proteines et sel pour 100g afin d'obtenir une évaluation du nutri-score</div>"))

out = interactive_output(find_nutri_score, {
    "fat": fat, 
    "carbohydrate": carbohydrate, 
    "fiber": fiber, 
    "protein": protein, 
    "salt": salt })

display(HBox([VBox([fat,carbohydrate, fiber, protein, salt]), out]))

HTML(value='<h1>Recherche de Nutri-score</h1>')

HTML(value="<div>Saisissez les Lipides, glucides, fibres, proteines et sel pour 100g afin d'obtenir une évalua…